# Github review stats

Purpose of this notebook is to have a better understanding the "reviewing situation".

- The purpose is NOT to blame and shame, but to inform.

## The general setup

In [ ]:
import requests
from dataclasses import dataclass
from collections import defaultdict


team_1 = ["0sewa0", "chrismuellner", "luhi-DT", "waodim", "gkrenn"]
team_2 = ["albertogdd", "andriisoldatenko", "aorcholski", "StefanHauth", "wepudt"]

@dataclass
class PRStat:
    number: int
    owner: str
    approvers: set[str]
    reviewers: set[str]

stats: list[PRStat] = list()

repo = 'dynatrace-operator'
owner = 'Dynatrace'


### Setup the auth

Needed so you don't run into the API limit.
You only need a read access to public repos, so nothing special.
Put you user/token into the `github.yaml`. (did this so no accidental token leak happens 😅)

In [ ]:
import yaml

user = ""
token = ""
with open("github.yaml") as stream:
    try:
        github = yaml.safe_load(stream)
        user = github["user"]
        token = github["token"]
    except yaml.YAMLError as exc:
        print(exc)

auth=(user, token)

### The settings for the Pull-Requests requests

In [2]:
pr_url = f'https://api.github.com/repos/{owner}/{repo}/pulls'
pr_params = dict()
pr_params["per_page"] = 100
pr_params["state"] = "closed"

## Make ALL the requests

In [3]:
for i in range(4):
    pr_params["page"] = i
    pr_response = requests.get(pr_url, params=pr_params, auth=auth)
    pulls = pr_response.json()
    for pull in pulls:
        pr_number = pull['number']
        pr_author = pull['user']
        if pr_author["login"] != "github-actions[bot]":
            rv_url = f'https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}/reviews'
            rv_params = dict()
            rv_params["per_page"] = 100
            rv_response = requests.get(rv_url, params=rv_params, auth=auth)
            reviews = rv_response.json()
            approvers:set[str] = set()
            reviewers:set[str] = set()
            for review in reviews:
                rv_author = review["user"]["login"]
                reviewers.add(rv_author)
                if review["state"] == "APPROVED":
                    approvers.add(rv_author)
            stat = PRStat(pr_number, pr_author, approvers, reviewers)
            stats.append(stat)

### "Analysis"

In [4]:
t1_reviews = 0
t2_reviews = 0
mixed_reviews = 0
t1_approves = 0
t2_approves = 0
mixed_approves = 0

approves = defaultdict(int)
reviews = defaultdict(int)

for stat in stats:
    is_rv_t1 = False
    is_rv_t2 = False
    for rv in stat.reviewers:
        reviews[rv] += 1
        if rv in team_1:
            is_rv_t1 = True
        if rv in team_2:
            is_rv_t2 = True
    if is_rv_t1 and not is_rv_t2:
        t1_reviews += 1
    elif is_rv_t2 and not is_rv_t1:
        t2_reviews += 1
    else:
        mixed_reviews += 1

    is_ap_t1 = False
    is_ap_t2 = False
    for ap in stat.approvers:
        approves[rv] += 1
        if ap in team_1:
            is_ap_t1 = True
        if ap in team_2:
            is_ap_t2 = True
    if is_ap_t1 and not is_ap_t2:
        t1_approves += 1
    elif is_ap_t2 and not is_ap_t1:
        t2_approves += 1
    else:
        mixed_approves += 1

print(f'Number of PRs looked at {len(stats)}')
print(f'PRs reviewed only by team-1 {t1_reviews}')
print(f'PRs reviewed only by team-2 {t2_reviews}')
print(f'PRs reviewed by both {mixed_reviews}')
print(f'PRs approved only by team-1 {t1_approves}')
print(f'PRs approved only by team-2 {t2_approves}')
print(f'PRs approved by both {mixed_approves}')

print(f'Approvers (1 per PR) {approves}')
print(f'Reviewers (1 per PR) {reviews}')

Number of PRs looked at 377
PRs reviewed only by team-1 57
PRs reviewed only by team-2 26
PRs reviewed by both 294
PRs approved only by team-1 72
PRs approved only by team-2 31
PRs approved by both 274
Approvers (1 per PR) defaultdict(<class 'int'>, {'albertogdd': 80, '0sewa0': 44, 'waodim': 67, 'aorcholski': 2, 'StefanHauth': 29, 'luhi-DT': 58, 'chrismuellner': 26, 'andriisoldatenko': 61, 'wepudt': 5})
Reviewers (1 per PR) defaultdict(<class 'int'>, {'waodim': 110, 'albertogdd': 43, 'aorcholski': 34, '0sewa0': 102, 'chrismuellner': 40, 'gkrenn': 13, 'StefanHauth': 28, 'luhi-DT': 44, 'andriisoldatenko': 82, 'wepudt': 6})
